<a href="https://colab.research.google.com/github/dcpetty/google-colaboratory/blob/main/mathematics/logicproof.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# `logicproof.py`

This is the [Collaboratory](https://colab.research.google.com/) notebook for [https://psb-david-petty.github.io/logicproof/](https://psb-david-petty.github.io/logicproof/).

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
#
# logicproof.py
#
# TODO: --verbose only allows for two
#
"""
logicproof.py takes zero or more premises and compares them with a goal.

http://intrologic.stanford.edu/glossary/operator_precedence.html

The premises and the goal must be in postfix form with tokens separated by
spaces to ease parsing, for example:

    a & b | c & d => e                  # which would be parenthesized as:
    ((a & b) | (c & d)) => e            # is written in postfix form as:
    a b & c d & | e =>
"""

__author__ = "David C. Petty"
__copyright__ = "Copyright 2020, David C. Petty"
__license__ = "https://creativecommons.org/licenses/by-nc-sa/4.0/"
__version__ = "0.0.2"
__maintainer__ = "David C. Petty"
__email__ = "david_petty@psbma.org"
__status__ = "Hack"

import argparse, os, re, sys


class Table:
    """Generate and print truth table for premises and goal."""
    _sep = '*'                              # table separator
    _emp = '+'                              # word emphasis
    _empty = ['T']                          # default empty premises
    _top = (0, lambda: 1)                   # true
    _nop = (1, lambda o: ~o & 1)            # not
    _aop = (2, lambda a, b: a & b)          # and
    _oop = (2, lambda a, b: a | b)          # or
    _iop = (2, lambda a, b: (~a | b) & 1)   # inference
    _bop = (2, lambda a, b: ~(a ^ b) & 1)   # biconditional
    # _ops is the dictionary of valid operation tuples.
    _ops = {'T': _top,                      # arity 0
            '~': _nop,                      # arity 1, arity 2
            '&': _aop, '|': _oop, '=>': _iop, '<=>': _bop, 'AND': _aop}

    def __init__(self, premises, goal, verbose=False):
        """Generate truth table for premises and goal."""
        self._premises = self._parse(premises)
        self._goals = self._parse(goal)
        assert len(self._goals) == 1, f"goal ({self._goals}) has > 1 item"
        self._verbose = verbose
        # Collect single-letter variable names from premises and goal, unless
        # premises is _empty.
        self._vars = set([t.lower() for exp in (
            self._premises if self._premises != self._empty else list())
            + self._goals for t in exp.split(' ')
            if t.lower() in 'abcdefghijklmnopqrstuvwxyz'])
        self._table = self._generate()

    """ ############### Table properties ############### """

    @property
    def premises(self):
        """Return self._premises list property."""
        return self._premises

    @property
    def goals(self):
        """Return self._goals list property."""
        return self._goals

    @property
    def table(self):
        """Return self._table multi-line string property."""
        return '\n'.join(self._table)

    @property
    def vars(self):
        """Return self._vars set property."""
        return self._vars

    @property
    def verbose(self):
        """Return self._verbose Boolean property."""
        return self._verbose

    """ ################ static methods ################ """

    @staticmethod
    def _parse(lines):
        """Return list of non-empty lines stripped of '#' comments."""
        return [
            (s[: s.find('#')] if s.find('#') >= 0 else s).strip()
            for s in lines.split('\n') if s
        ]

    @staticmethod
    def _values(n, var):
        """Return value dict for reversed sorted var list.
        For example, an n of 3 might yield {'a': 0, 'b': 1, 'c': '1'}"""
        assert 0 <= n < 2 ** len(var), f"{n} not on [0, 2 ** len({var}))"
        result = dict()
        for i, v in enumerate(reversed(sorted(var))):
            result[v] = (1 << i & n) >> i
        return result

    """ ############### instance methods ############### """

    def _evaluate_premises(self, n):
        """Return _evaluate of AND-conjoined premises for vars encoded w/ n."""
        return self._evaluate(
            ' '.join(self.premises + ['AND'] * (len(self.premises) - 1)),
            self._values(n, self.vars))

    def _evaluate_goal(self, n):
        """Return _evaluate of one-and-only goal for vars encoded w/ n"""
        return self._evaluate(self.goals[0], self._values(n, self.vars))

    def _markdown_encode(self, s):
        """Encode logic expressions for markdown table. In particular, the
        discunction symbol '|' is the same as the table cell delimiter."""
        an, aa, ao, ai, ab = '~', '&', '|', '=>', '<=>'
        un, ua, uo, ui, ub = '&not;', '&and;', '&or;', '&rArr;', '&hArr;'
        un, ua, uo, ui, ub = '\u00AC', '\u2227', '\u2228', '\u21D2', '\u21D4'
        ms, me = ' | ', '**'
        # ab must be replaced before ai, self._sep must be replaced after ao,
        # and self._emp must be replaced last.
        return s.replace(an, un)\
            .replace(aa, ua) \
            .replace(ao, uo) \
            .replace(ab, ub) \
            .replace(ai, ui) \
            .replace(self._sep, ms) \
            .replace(self._emp, me).strip()

    def _header(self):
        """Create two-line header for table."""
        dvb, nl, sep, wsq = '\u2016', '\n', self._sep, '\u25a1'
        vnames = f"{''.join(sorted(self.vars))}"
        pexp, pval = zip(*self._evaluate_premises(0))
        gexp, gval = zip(*self._evaluate_goal(0))
        header = self._markdown_encode(
            f"{sep}{vnames}{sep}{dvb}{sep}{sep.join(pexp)}{sep}" \
            f"{wsq}{sep}{sep.join(gexp)}{sep}")
        return header + '\n' \
            + f"{'| :---: ' * (len([c for c in header if c == '|']) - 1)}|"

    def _line(self, n):
        """Format table line."""
        dvb, sep, wsq = '\u2016', self._sep, '\u25a1'
        values, b = self._values(n, self.vars), lambda n: 'T' if n else 'F'
        pexp, pval = zip(*self._evaluate_premises(n))
        gexp, gval = zip(*self._evaluate_goal(n))
        number = \
            f"+{''.join(b(v) for v in values.values())[::-1]}+"
        line = self._markdown_encode(
            f"{sep}{number}{sep}{dvb}{sep}{sep.join(b(v) for v in pval)}{sep}" \
            f"{wsq}{sep}{sep.join(b(v) for v in gval)}{sep}")
        # TODO:
        if pval[-1]:
            return line.replace(wsq, '\u25a0')
        elif self.verbose:
            return line

    def _evaluate(self, exp, values):
        """Return (full_expression, evaluated_value, ) tuple that is result
        of evaluating exp in the context of values."""
        stack, calcs, sregex = list(), list(), re.compile(r'\s*')
        if exp in 'abcdefghijklmnopqrstuvwxyz':
            return [(exp, values[exp], )]
        for t in exp.split(' '):
            if t in 'abcdefghijklmnopqrstuvwxyz':
                # If t is a variable, push variable and value.
                stack.append((t, values[t],))
            elif t in self._ops:
                # If t is an op, evaluate push expression, push it and value.
                num, op = self._ops[t]  #
                pops = stack[-num: ]    #
                del stack[-num: ]       #
                exps, vals = zip(*pops) if pops else (list(), list(), )
                exp, val = ' '.join(list(exps) + [t]), op(*vals)
                stack.append((exp, val, ))
                calcs.append((exp, val,))
            else:
                print(f"unknown token: {t}")
        assert len(stack) == 1, f"unbalanced stack ({stack})"
        return calcs

    def _generate(self):
        # Evaluate all dicts.
        lines = [self._line(n) for n in range(2 ** len(self.vars))]
        return [self._header()] + [line for line in lines if line]


class Parser(argparse.ArgumentParser):
    """Create OptionParser to parse command-line options."""
    def __init__(self, **kargs):
        argparse.ArgumentParser.__init__(self, **kargs)
        # self.remove_argument("-h")
        self.add_argument("-?", "--help", action="help",
                          help="show this help message and exit")
        self.add_argument('--version', action='version',
                          version=f"%(prog)s {__version__}")

    def error(self, msg):
        sys.stderr.write("%s: error: %s\n\n" % (self.prog, msg, ))
        self.print_help()
        sys.exit(2)


def main(argv):
    """Collect command-line arguments, create Table, and print it."""
    description = """Collect one or more premises and a goal and display a
    truth table whose entries are every combination of inputs where all
    premises are True."""
    formatter = lambda prog: \
        argparse.ArgumentDefaultsHelpFormatter(prog, max_help_position=30)
    parser = Parser(description=description, add_help=False,
                    formatter_class=formatter)
    arguments = [
        # c1, c2, action, dest, default, help
        ('-e', '--echo', 'store_true', 'ECHO', False,
         'echo premise(s) and goal',),
        ('-v', '--verbose', 'store_true', 'VERBOSE', False,
         'echo entire truth table',),
    ]
    # Add optional arguments with values.
    for c1, c2, a, v, d, h in arguments:
        parser.add_argument(c1, c2, action=a, dest=v, default=d, help=h)
    # Add positional arguments. 'NAME' is both the string and the variable.
    parser.add_argument('PREMISE', nargs='*', help='zero or more premises')
    parser.add_argument('GOAL', help='goal to match to premise(s)')
    # Parse arguments.
    pa = parser.parse_args(args=argv[1: ])
    if not pa.PREMISE:
        pa.PREMISE = Table._empty
    if pa.ECHO:
        if pa.PREMISE:
            space, sq = '\n' + 11 * ' ', "'"
            print(f"PREMISES = " \
                  f"{f'{space}'.join([f'{sq}{p}{sq}' for p in pa.PREMISE])}")
        if pa.GOAL:
            print(f"    GOAL = '{pa.GOAL}'")
    # Create the truth table.
    table = Table('\n'.join(pa.PREMISE), pa.GOAL, pa.VERBOSE)
    # TODO: this simply prints the table
    print(table.table)


if __name__ == '__main__':
    is_idle, is_pycharm, is_jupyter = (
        'idlelib' in sys.modules,
        int(os.getenv('PYCHARM', 0)),
        '__file__' not in globals()
    )
    if any((is_idle, is_pycharm, is_jupyter,)):
        main(["logicproof.py",
              "--help", ])
    else:
        main(sys.argv)


usage: ipykernel_launcher.py [-?] [--version] [-e] [-v]
                             [PREMISE [PREMISE ...]] GOAL

Collect one or more premises and a goal and display a truth table whose
entries are every combination of inputs where all premises are True.

positional arguments:
  PREMISE        zero or more premises (default: None)
  GOAL           goal to match to premise(s)

optional arguments:
  -?, --help     show this help message and exit
  --version      show program's version number and exit
  -e, --echo     echo premise(s) and goal (default: False)
  -v, --verbose  echo entire truth table (default: False)


SystemExit: ignored

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


## Evaluate proofs from the [virtual classroom](https://docs.google.com/presentation/d/e/2PACX-1vTg06S2eC-4g-GdnkG7_IzAX-ByUSugc_RlwGVb7FaEFgpVHH8L7rJkPGIZ_LqRmm0r2rqNMciuH3jX/pub?slide=id.p)

- Activities [4.1](https://onedrive.live.com/?authkey=%21AKEIaMjOI4GZ8gU&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156158&parId=477D5CE9111F9830%2155559)
- Activities [4.2](https://onedrive.live.com/?authkey=%21AJj2NI3eziQneuA&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156159&parId=477D5CE9111F9830%2155559)
- Activities [4.3](https://onedrive.live.com/?authkey=%21AL2JfVYrc8l6-NE&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156368&parId=477D5CE9111F9830%2155559)
- Activities [4.4]()

In [ ]:
activities = [
    # Activity 4.1 from the virtual classroom:
    # https://onedrive.live.com/?authkey=%21AKEIaMjOI4GZ8gU&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156158&parId=477D5CE9111F9830%2155559
    [   "logicproof.py", "-e",
        "w q r & => # w => (q&r)",
        "u w p & => # u => (w&p)",
        "s u & # s&u",
        "q a | # q|a",
    ],
    [   "logicproof.py", "-e",
        "p # p",
        "p q r & => # p =>(q&r)",
        "s r <=> # s<=>r",
        "s # s",
    ],
    [   "logicproof.py", "-e",
        "q p & # (q&p)",
        "q s | w => # (q|s) =>w",
        "r w v | <=> # r <=> (w|v)",
        "r # r",
    ],
    # Activity 4.2 from the virtual classroom:
    # https://onedrive.live.com/?authkey=%21AJj2NI3eziQneuA&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156159&parId=477D5CE9111F9830%2155559
    [   "logicproof.py", "-e",
        "a b & p <=> # (a&b) <=> p",
        "b s <=> # b <=> s",
        "p s => # p => s",
    ],
    [   "logicproof.py", "-e",
        "w p q & => # w => (p&q)",
        "s q r | <=> # s <=> (q|r)",
        "w s => # w => s",
    ],
    [   "logicproof.py", "-e",
        "r p q & => # r =>(p&q)",
        "s p w | <=> # s <=> (p|w)",
        "r s => # r => s",
    ],
    [   "logicproof.py", "-e",
        "r ~ s <=> # ~r<=>s",
        "q r ~ <=> # q<=>~r",
        "q s ~ => # q => ~s",
        "r # r",
    ],
    [   "logicproof.py", "-e",
        "p # p",
        "r q => # r=>q",
        "w s <=> # w<=>s",
        "r s | # r|s",
        "w q => # w=>q",
        "p q & # p&q",
    ],
    # Activity 4.3 from the virtual classroom:
    # https://onedrive.live.com/?authkey=%21AL2JfVYrc8l6-NE&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156368&parId=477D5CE9111F9830%2155559
    [   "logicproof.py", "-e",
        "s r &       # (S&R)",
        "a b s ~ & =># A=>(B&~S)",
        "a ~         # ~A",
    ],
    [   "logicproof.py", "-e",
        "a b &       # (a&b)",
        "p q a ~ & =># p=>(q&~a)",
        "p ~         # ~p",
    ],
    [   "logicproof.py", "-e",
        "r s w & =>      # r => ( s & w )",
        "w r <=>         # w <=> r",
        "r ~ ~           # ~~r",
        "w q =>          # w => q",
        "s q & r <=>     # ( s & q ) <=> r",
    ],
    [   "logicproof.py", "-e",
        "r m & s w & =>         # (r&m)=>(s&w)",
        "r m & a & s w & b | => #  (r&m&a)=>(s&w|b)",
    ],
    # Activity 4.4 from the virtual classroom:
    # https://onedrive.live.com/?authkey=%21ALcQGP2kXq6u8uw&cid=477D5CE9111F9830&id=477D5CE9111F9830%2156552
    [   "logicproof.py", "-e",
        "p q =>         # p=>q",
        "p q |          # m=> (p|q)",
        "m q =>         # m=>q",
    ],
    [   "logicproof.py", "-e",
        "p q r => =>    # (p=>(q=>r)",
        "p q => p r => =># (p=>q) => (p=>r)",
    ],
    [   "logicproof.py", "-e",
        "p q p => =>    # p=>(q=>p)",
    ],
    [   "logicproof.py", "-e",
        "p q =>         # p=>q",
        "s r <=>        # s<=>r",
        "s q =>         # s=>q",
        "m s =>         # m=>s",
        "p r | m |      #  p|r|m",
        "q m r s & <=> |# (q | (m<=>(r&s)))",
    ],
    [   "logicproof.py", "-e",
        "r s q p & | |  # (r|s|(q&p))",
        "m r <=>        # m<=>r",
        "q s <=>        # q<=>s",
        "q m =>         # q=>m",
        "m              # m",
    ],
    [   "logicproof.py", "-e",
        "p q |  # (p|q)",
        "p ~    # ~p",
        "q      # q",
    ],
    [   "logicproof.py", "-e",
        "p q => # p=>q",
        "p ~ q |# (~p|q)", "-e"
    ],
]

# Print truth tables for all activities.
print("# Print truth tables for all activities.")
for activity in activities:
    main(activity)

# Print truth tables for all activities with '--verbose' flag.
print("# Print truth tables for all activities with '--verbose' flag.")
for activity in activities:
    main(activity + ['-v'])


# Print truth tables for all activities.
PREMISES = 'w q r & => # w => (q&r)'
           'u w p & => # u => (w&p)'
           's u & # s&u'
    GOAL = 'q a | # q|a'
| apqrsuw | ‖ | q r ∧ | w q r ∧ ⇒ | w p ∧ | u w p ∧ ⇒ | s u ∧ | u w p ∧ ⇒ s u ∧ AND | w q r ∧ ⇒ u w p ∧ ⇒ s u ∧ AND AND | □ | q a ∨ |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **FTTTTTT** | ‖ | T | T | T | T | T | T | T | ■ | T |
| **TTTTTTT** | ‖ | T | T | T | T | T | T | T | ■ | T |
PREMISES = 'p # p'
           'p q r & => # p =>(q&r)'
           's r <=> # s<=>r'
    GOAL = 's # s'
| pqrs | ‖ | q r ∧ | p q r ∧ ⇒ | s r ⇔ | p q r ∧ ⇒ s r ⇔ AND | p p q r ∧ ⇒ s r ⇔ AND AND | □ | s |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| **TTTT** | ‖ | T | T | T | T | T | ■ | T |
PREMISES = 'q p & # (q&p)'
           'q s | w => # (q|s) =>w'
           'r w v | <=> # r <=> (w|v)'
    GOAL = 'r # r'
| pqrsvw | ‖ | q p ∧ | q s ∨ | q s ∨ w ⇒ | w v ∨ | r w v 